In [1]:
!pip install langchain faiss-cpu python-dotenv huggingface_hub PyMuPDF langchain-community pypdf transformers accelerate

In [18]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [17]:
# Importa as bibliotecas
import os
from dotenv import load_dotenv, find_dotenv
from google.colab import files

# Carrega variáveis de ambiente (arquivo .env)
load_dotenv()

# Definindo a variável de ambiente do HF
# Permite que o Langchain acesse o token
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [18]:
# Importando bibliotecas
from langchain.document_loaders import PyPDFLoader # Carregador de PDFs
from langchain.text_splitter import RecursiveCharacterTextSplitter # Divisor de texto

# Caminho para o PDF
caminho_pdf = "/content/chat-docs-rag/data/documentos/contrato_ficticio.pdf"

# Carrega o PDF
# Cada página vira um objeto 'Document' separado
loader = PyPDFLoader(caminho_pdf)
documentos = loader.load()

# Exibe a quantidade de páginas e o conteúdo da primeira página
print(f"Total de páginas: {len(documentos)}")
print("\nConteúdo da primeira página:\n")
print(documentos[0].page_content)

# Define o divisor de texto
divisor = RecursiveCharacterTextSplitter(
    chunk_size = 500,         # Tamanho máximo de cada chunk
    chunk_overlap = 50)       # Sobreposição para preservar contexto entre cada pedaço

# Divide os documentos em pedaços menores
chunks = divisor.split_documents(documentos)

# Mostra o chunk dividido
print(f"\nTotal de chunks gerados: {len(chunks)}")
print("\nExemplo de chunk:\n")
print(chunks[0].page_content)

Total de páginas: 2

Conteúdo da primeira página:

CONTRATO DE PRESTAÇÃO DE SERVIÇOS
Pelo presente instrumento, as partes abaixo identificadas:
CONTRATANTE: Empresa XYZ Ltda, CNPJ 00.000.000/0001-00
CONTRATADA: Leonardo Thomaz Marinsek, CPF 000.000.000-00
Cláusula 1 - Objeto
O presente contrato tem por objeto a prestação de serviços de consultoria em tecnologia da
informação, com foco em soluções baseadas em inteligência artificial.
Cláusula 2 - Vigência
O contrato terá duração de 12 (doze) meses, com início em 01/06/2024 e término em 31/05/2025.
Cláusula 3 - Remuneração
A contratada receberá o valor mensal de R$ 8.000,00 (oito mil reais), pagos até o quinto dia útil de
cada mês.
Cláusula 4 - Rescisão
O contrato poderá ser rescindido por qualquer das partes, mediante aviso prévio de 30 (trinta) dias.
Em caso de descumprimento, será aplicada multa de 10% sobre o valor total restante do contrato.
Cláusula 5 - Foro
Para dirimir quaisquer controvérsias oriundas deste contrato, as partes el

In [19]:
# Importa os componentes
from langchain.embeddings import HuggingFaceEmbeddings # Embeddings
from langchain.vectorstores import FAISS # Armazenamento vetorial

# Define o diretório onde o índice vetorial será salvo
diretorio_faiss = "faiss_index"

# Usa o modelo da Hugging Face pela API, sem instalar localmente
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Cria o banco vetorial FAISS com os chunks e os embeddings gerados
vetor_db = FAISS.from_documents(chunks, embeddings)

# Salva o banco vetorial localmente
vetor_db.save_local(diretorio_faiss)

print("✔️ Vetores gerados e salvos com sucesso.")

✔️ Vetores gerados e salvos com sucesso.


In [20]:
# Importa os componentes
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.chains import RetrievalQA

# Define o modelo LLM da Hugging Face
model_id = "google/flan-t5-large"

# Carrega o tokenizer e modelo
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Cria o pipeline de geração de texto
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    #max_length=512,
    #truncation=True
)

# Conecta o pipeline ao Langchain
llm = HuggingFacePipeline(pipeline=pipe)

# Cria o mecanismo de busca baseado em similaridade semântica
retriever = vetor_db.as_retriever()

# Cria a cadeia RAG: recuperação + geração de resposta
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True # Retorna os chunks usados
)

# Pergunta para testar o modelo
pergunta = "Qual é a cláusula de rescisão do contrato?"
resposta = qa_chain.invoke({"query": pergunta})

# Mostra a resposta gerada
print("📌 Resposta da IA:\n")
print(resposta["result"])

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (582 > 512). Running this sequence through the model will result in indexing errors


📌 Resposta da IA:

aviso previo de 30 (trinta) dias
